In [8]:
%pip install torch
%pip install torchvision

     ---------------------------------------- 1.1/1.1 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [98]:
%pip install --upgrade numpy

     ---------------------------------------- 14.8/14.8 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from PIL import Image
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from skimage import io
import os

In [42]:
pathToCategories = "./baseline/baseline/"
# labels=["highway", "office", "pedestrians", "PETS2006"]
# fonction d'acquisition des donnees dentrees et de validation pour l'analyse par rapport a une categories de video mis en parametre
def init_dataset(labels):
    input_data = {}
    groundTruth_data = {}
    nb_image = {}
    i=0
    for label in labels:
        input_data[label] = [str(p) for p in Path(
            pathToCategories+label+"/input/").glob('*.jpg')]
        groundTruth_data[label] = [str(p) for p in Path(
            pathToCategories+label+"/groundtruth/").glob('*.png')]
        nb_image[label] = len(input_data[label])
    return nb_image,input_data,groundTruth_data


In [31]:
input_data,groundTruth_data=init_dataset(labels,4)

TypeError: init_dataset() takes 1 positional argument but 2 were given

### Segmentation sémantique

In [43]:
#fonction appliquant des pretransformation aux cathegories dimage recu et retournant la liste des images retransforme en Tensor

preprocessing = transforms.Compose([
    transforms.ToTensor(),
])
# def get_input_tensor(labels,input_data):
#     input_tensor_label = {}
#     for label in labels:
#         input_tensor_label[label] = []
#         for input in input_data[label]:
#             tensor = preprocessing(Image.fromarray(io.imread(input)))#io.imread(os.path.join(input)) )
#             #tensor = preprocessing(cv2.imread("./"+input))
#             input_tensor_label[label].append(tensor)
#     return input_tensor_label
def get_input_tensor(input_data):
    input_tensor_label = {}
    input_tensor_label = []
    for input in input_data:
        tensor = preprocessing(Image.fromarray(io.imread(input)))#io.imread(os.path.join(input)) )
            #tensor = preprocessing(cv2.imread("./"+input))
        input_tensor_label.append(tensor)
    return input_tensor_label

In [26]:
input_tensor_label=get_input_tensor(labels,input_data)

KeyboardInterrupt: 

In [45]:
# return images in batch for prediction analysist
#def stack_input_tensor(labels,input_tensor_label):
    #input_batch_tensor_label = {}
    #for label in labels:
        #input_batch_tensor_label[label] =[]
        #for tensor in input_tensor_label[label]:
        #input_batch_tensor_label[label]=torch.stack(input_tensor_label[label], 0)
        #input_batch_tensor_label=torch.stack(input_tensor_label, 0)
        #input_batch_tensor_label[label] = input_batch_tensor_label[label].cuda() 
    #return input_batch_tensor_label

def stack_input_tensor(input_tensor_label):
    input_batch_tensor_label=torch.stack(input_tensor_label, 0)
    return input_batch_tensor_label


In [47]:
def evaluate(input_batch_tensor_label):
    model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()
    predictions = {}
    with torch.no_grad():
        predictions = model(input_batch_tensor_label)
    return predictions

In [78]:
def draw_segmentation_mask(image):
  name_colors=list(mcolors.CSS4_COLORS.keys())[7:7+2]
  rgb_colors=np.array([(np.array(mcolors.to_rgb(i))*255).astype("uint8") for i in name_colors])
  image.putpalette(rgb_colors)
  return image

## Fonction Main de lexecution pour obtenir la segmentation sementique

In [82]:
labels = ["highway"]# "office", "pedestrians", "PETS2006"]
batchSize = 4
print("initialising dataset: ",labels," with batchSize = ",batchSize)
nb_image, input_data, true_data = init_dataset(labels)
mini_input_data={}
for label in labels:
    print("current label :",label)
    current = 0
    anounce_ID =250
    while(current < nb_image[label]):
        if current > anounce_ID:
            print("are now at", anounce_ID)
            anounce_ID +=250
        if (current+batchSize) > nb_image[label]:
            mini_input_data[label]=input_data[label][current:-1]
            current = nb_image[label]
        else:
            mini_input_data[label]=input_data[label][current:(current+batchSize)]
            current += batchSize

        input_tensor_label = get_input_tensor(mini_input_data[label])
        input_batch_tensor_label = stack_input_tensor(input_tensor_label)
        predictions = evaluate(input_batch_tensor_label)
        for i in range(len(mini_input_data[label])):
            prediction = torch.argmax(predictions['out'][i].squeeze(), dim=0).detach().cpu().numpy().astype(np.uint8)
            masks = Image.fromarray(prediction)
            segmentation_mask=draw_segmentation_mask(masks)
            segmentation_mask.save(mini_input_data[label][i].replace("input","output_sem_seg"),format="PNG")



initialising dataset:  ['highway']  with batchSize =  4
current label : highway
are now at 250
are now at 500
are now at 750
are now at 1000
are now at 1250
are now at 1500
